<a href="https://colab.research.google.com/github/maverick98/CDS/blob/main/image_caption_transformer_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import necessary libraries

In [1]:
import numpy as np
import string
import tensorflow as tf
from tensorflow import math, matmul, reshape, shape, transpose, cast, float32
from tensorflow.data import Dataset
from tensorflow.keras.layers import Embedding,Dense, Layer,TextVectorization
from tensorflow.keras.backend import softmax
from pickle import load, dump, HIGHEST_PROTOCOL
from sklearn.utils import shuffle
from numpy import savetxt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import convert_to_tensor, int64
from tensorflow.keras.layers import LayerNormalization,   ReLU, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import LearningRateSchedule
from tensorflow.keras.metrics import Mean
from tensorflow import data, train,  reduce_sum,  equal, argmax,GradientTape, function
from tensorflow.keras.losses import sparse_categorical_crossentropy
from time import time
from pickle import dump
from tensorflow import    linalg, ones, maximum, newaxis
from tensorflow.keras import Model
from pickle import load
from tensorflow import Module
from tensorflow import   TensorArray, argmax,  transpose
from matplotlib.pylab import plt
from numpy import arange

from keras.utils import to_categorical
import numpy as np
from numpy import array
import pandas as pd
import cv2
from glob import glob
import PIL
import time
from tqdm import tqdm
import os
#import gensim
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import sentence_bleu
from PIL import Image
import random
from importlib.machinery import SourceFileLoader
from os.path import join
from google.colab import drive

# Mount Google Drive

In [2]:


ROOT = "/content/drive"
drive.mount(ROOT)
PROJ = "MyDrive/Capstone/src" 
PROJECT_PATH = join(ROOT, PROJ)
!mkdir -p "{PROJECT_PATH}"



Mounted at /content/drive


In [3]:
PROJECT_PATH

'/content/drive/MyDrive/Capstone/src'

# Check if connected to GPU

In [13]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


# Load Modules

In [14]:
def load_module(module_name):
    module_py=module_name+'.py'
    SourceFileLoader(module_name, join(PROJECT_PATH, module_py)).load_module()
    


In [32]:
load_module('transformer_model_params')
load_module('transformer_training_params')
load_module('positional_encoding')

# Create Transformer Model and Training params

In [34]:
h=8 #Number of self-attention heads
d_k=64 #Dimensionality of the linearly projected queries and keys
d_v=64 #Dimensionality of the linearly projected values
d_model=512 #Dimensionality of the model layer's outputs
d_ff=2048 #Dimensionality of the inner FC Layer
num_layers=6 #No of layers to repeat for both EncoderLayer and DecoderLayer

from transformer_model_params import TransformerModelParams
transformerModelParams=TransformerModelParams()
transformerModelParams.set_h(h)\
                 .set_d_k(d_k)\
                 .set_d_v(d_v)\
                 .set_d_model(d_model)\
                 .set_d_ff(d_ff)\
                 .set_num_layers(num_layers)

In [35]:
epochs=8 #Number of epochs
d_k=64 #Dimensionality of the linearly projected queries and keys
batch_size=64 #Batch size
beta_1=0.9 # beta_1 of ADAM
beta_2=0.98 # beta_2 of ADAM
epsilon=1e-9 #Epsilon
dropout_rate=0.1 #Dropout rate 

from transformer_training_params import TransformerTrainingParams
transformerTrainingParams=TransformerTrainingParams()
transformerTrainingParams.set_epochs(epochs)\
                 .set_batch_size(batch_size)\
                 .set_beta_1(beta_1)\
                 .set_beta_2(beta_2)\
                 .set_epsilon(epsilon)\
                 .set_dropout_rate(dropout_rate)

# Check Word Embedding and  Positional Encoding Layer in Transformers

In [ ]:
output_sequence_length=4
vocab_size=5
sentences=[["I am good"],["She is good"],["All are good"]]
sentence_data=Dataset.from_tensor_slices(sentences)
vectorize_layer=TextVectorization(output_sequence_length=output_sequence_length,max_tokens=vocab_size)
vectorize_layer.adapt(sentence_data)
word_tensors=convert_to_tensor(sentences,dtype=tf.string)
vectorized_words=vectorize_layer(word_tensors)
print("Vocabulary: ",vectorize_layer.get_vocabulary())
print("Vectorized words : ",vectorized_words)

In [ ]:
output_length=6
word_embedding_layer=Embedding(vocab_size,output_length)
embedded_words=word_embedding_layer(vectorized_words)
print("embedded words are ",embedded_words)

In [ ]:
position_embedding_layer=Embedding(output_sequence_length,output_length)
position_indices=tf.range(output_sequence_length)
embedded_position_indices=position_embedding_layer(position_indices)
print("embedded_position_indices are ",embedded_position_indices)

In [ ]:
final_output_embedding=embedded_words+embedded_position_indices
print("final_output_embedding ",final_output_embedding)